In [60]:
import csv
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json
import time
from datetime import datetime

In [8]:
url = "search-open-search-hello-world-ptsefi3fh6osw2up7h6mq3ai3a.us-east-1.es.amazonaws.com"

In [9]:
region = 'us-east-1'
service = 'es'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

In [52]:
client = OpenSearch(hosts = [{'host': url, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20)

# Create index

In [88]:
index_name = 'food-reviews-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 4
    }
  }
}

response = client.indices.create(index_name, body=index_body)
response

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'food-reviews-index'}

In [5]:
! head -2 Reviews.csv

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [87]:
client.indices.delete(
    index = index_name
)

{'acknowledged': True}

In [90]:
import math

In [ ]:
def split_into_batches(array, max_batch_size):
    batch = []
    for elem in array:
        batch.append(elem)
        if len(batch) == max_batch_size:
            yield batch
            batch = []

    if len(batch) > 0:
        yield batch

In [89]:
%%time
def generate_bulk_data(index_name, reader, batch_no, batch_size):
    all_data = []
    data = []
    for i, row in enumerate(reader, start=1):
        data.append(
            { "index": {
            "_index": index_name,
            "_id": row.get('Id', None),  # if you don't have an 'id' field, you can remove this line
        }
            }
        )
        data.append(row)
        if i % batch_size == 0:
            all_data.append(data)
            data = []
        # if i % 10000 == 0:
        #     break
    all_data.append(data)
    return "\n".join([json.dumps(doc) for doc in data])


with open('Reviews.csv', 'r') as f:
    reader = csv.DictReader(f)
    data = generate_bulk_data(index_name, reader)
    success = client.bulk(data)
    print(f'Took: {success["took"]}ms,  Errors: {success["errors"]}')


ConnectionError: ConnectionError(('Connection aborted.', TimeoutError('The write operation timed out'))) caused by: ConnectionError(('Connection aborted.', TimeoutError('The write operation timed out')))

In [ ]:
print("A")

In [21]:
q = '5'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['Score']
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)

In [22]:
response

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 0.6931471,
  'hits': [{'_index': 'food-reviews-index',
    '_id': '1',
    '_score': 0.6931471,
    '_source': {'Id': '1',
     'ProductId': 'B001E4KFG0',
     'UserId': 'A3SGXH7AUHU8GW',
     'ProfileName': 'delmartian',
     'HelpfulnessNumerator': '1',
     'HelpfulnessDenominator': '1',
     'Score': '5',
     'Time': '1303862400',
     'Summary': 'Good Quality Dog Food',
     'Text': 'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'}},
   {'_index': 'food-reviews-index',
    '_id': '9',
    '_score': 0.6931471,
    '_source': {'Id': '9',
     'ProductId': 'B000E7L2R4',
     'UserId': 'A1MZYO9TZK0BBI',
 